## InfluxDB ##

In [1]:
from influxdb import InfluxDBClient
import json

In [2]:
client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)

In [3]:
dbs= client.get_list_database()

In [4]:
client.switch_database('prices')

In [5]:
measurements = client.get_list_measurements()

In [6]:
results = client.query('select * from ALC_allcoin ORDER by time DESC LIMIT 1')
results.raw

{'statement_id': 0,
 'series': [{'name': 'ALC_allcoin',
   'columns': ['time', 'price_btc', 'price_usd'],
   'values': [['2018-10-30T09:12:51Z', 2.31e-06, 0.0145388019]]}]}

In [7]:
str = str(measurements)

In [8]:
str = str.replace("\'", "\"")

In [9]:
raw_input = json.loads(str)

In [10]:
output_dict = [x for x in raw_input if x['name'] == 'ALC_allcoin']

In [11]:
output_dict

[{'name': 'ALC_allcoin'}]

In [12]:
clear_USD = [x for x in measurements 
               if x['name'].endswith('_USD') == False
              ]

In [13]:
clear_BTC = [x for x in clear_USD 
               if x['name'].endswith('_BTC') == False
              ]

In [14]:
clear_BTC1 = [x for x in clear_BTC 
               if x['name'].endswith('coin') 
              ]

In [61]:
a = points[1]
type(a)
a

{'time': '2018-10-28T20:43:56Z',
 'price_btc': 2.28e-06,
 'price_usd': 0.0147648444}

In [12]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" WHERE time >= now() - 1d LIMIT 100') %  x['name'])
    points = list(results.get_points())
    #points[0]['name'] = x['name']
    for p in points:
        p['name'] = x['name']
    #print(points)
      


In [13]:
points

[{'time': '2018-10-29T09:22:56Z',
  'price_btc': 2.28e-06,
  'price_usd': 0.0146646949,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:27:44Z',
  'price_btc': 2.28e-06,
  'price_usd': 0.014679866,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:32:50Z',
  'price_btc': 2.12e-06,
  'price_usd': 0.0136287905,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:36:53Z',
  'price_btc': 2.11e-06,
  'price_usd': 0.0136454911,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:41:54Z',
  'price_btc': 2.12e-06,
  'price_usd': 0.0136522128,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:49:52Z',
  'price_btc': 2.13e-06,
  'price_usd': 0.0136724272,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T09:55:56Z',
  'price_btc': 2.11e-06,
  'price_usd': 0.0136443575,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T10:00:54Z',
  'price_btc': 2.12e-06,
  'price_usd': 0.0136434272,
  'name': 'ALC_allcoin'},
 {'time': '2018-10-29T10:04:56Z',
  'price_btc': 2.11e-06,
  'price_usd': 0.0136229351,
 

In [23]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1000') %  x['name'])
    points = list(results.get_points())
    for i, p in enumerate(points):
        #print('index %d, value %s' % (i, p)) 
        points[i]['name'] = x['name']
        producer.send_messages('test', json.dumps(points[i], default=json_util.default).encode('utf-8'))


In [15]:
out = json.dumps(results.raw)
print(out)

{"statement_id": 0, "series": [{"name": "ALC_allcoin", "columns": ["time", "price_btc", "price_usd"], "values": [["2018-10-28T19:25:54Z", 2.28e-06, 0.01474555]]}]}


In [28]:
print(output_dict)

[{'name': '611_USD'}]


## Working with JSON output ##

In [21]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    
    points = list(results.get_points())
    print(points)
    data_file = x['name']+'.json'
    with open(data_file, 'w') as outfile:
        #json.dump(points, outfile)
        producer.send_messages('test', json.dumps(points, default=json_util.default).encode('utf-8'))
        print(x['name']+' message sent')
# 611_USD.json
#results.raw
#print(points)

[{'time': '2018-10-28T19:25:38Z', 'marketcap': 58289, 'price': 0.1267165628, 'volume': 23}]
611_USD message sent


In [37]:
results.raw    

{'statement_id': 0,
 'series': [{'name': '611_USD',
   'columns': ['time', 'marketcap', 'price', 'volume'],
   'values': [['2018-10-22T19:35:33Z', 65116, 0.1417095519, 122]]}]}

In [188]:
# save relusts.raw in a json file
data_file = output_dict['name']
with open('data.json', 'w') as outfile:
    json.dump(results.raw, outfile)

In [38]:
# save columns and values in a json file
with open('data.json', 'w') as outfile:
    json.dump(points, outfile)

In [131]:
def read_data(measurements):
    client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)
    client.switch_database('prices')
    
    for x in measurements:
        results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
        points = list(results.get_points())
        #params = points[0]
        #params['name'] = x['name']
        
        
        points[0]['name'] = x['name']
        #producer.send_messages('test', json.dumps(points[0], default=json_util.default).encode('utf-8'))

        

        #data_file = x['name']+'.json'
        #with open(data_file, 'w') as outfile:
            #json.dump(points, outfile)
    return points[0]
    #return params

In [132]:
read_data(output_dict)


{'time': '2018-10-28T20:58:37Z',
 'marketcap': 57960,
 'price': 0.1259996609,
 'volume': 23,
 'name': '611_USD'}

In [118]:
type(x)

NoneType

In [106]:
print(x.keys)

<built-in method keys of dict object at 0x0128BDE0>


In [61]:
json.dumps(x)

'[{"time": "2018-10-28T20:07:33Z", "marketcap": 57928, "price": 0.1259316913, "volume": 23, "name": "611_USD"}]'

## Kafka ##

In [19]:
from kafka import SimpleProducer, KafkaClient


kafka = KafkaClient('blockchain-kafka-kafka.default.svc.cluster.local:9092')
producer = SimpleProducer(kafka)
#producer.send_messages('test', b'async message')

In [21]:
import yaml
import json
from bson import json_util


In [22]:
import kafka as k
k.__version__

'1.4.3'

In [4]:
with open('611_USD.json') as f:
    data = json.load(f)

In [5]:
print(data)

[{'time': '2018-10-22T19:46:29Z', 'marketcap': 65429, 'price': 0.1423885136, 'volume': 123}]


In [22]:
with open('611_USD.json') as f:
    for line in f:
        d = yaml.safe_load(line)
        jd = json.dumps(d)
       
        producer.send_messages('test', json.dumps(jd, default=json_util.default).encode('utf-8'))


In [8]:
# send 10 messages to kafka
for i in range(10):
    data = { 'tag ': 'blah',
        'name' : 'sam',
        'index' : i,
        'score': 
            {'row1': 100,
             'row2': 200
        }
    }   
    producer.send_messages('test', json.dumps(data, indent=4, default=json_util.default).encode('utf-8'))

In [ ]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    
    points = list(results.get_points())
    print(points)
    data_file = x['name']+'.json'
    with open(data_file, 'w') as outfile:
        #json.dump(points, outfile)
        #producer.send_messages('test', json.dumps(points, default=json_util.default).encode('utf-8'))
        print(x['name']+' message sent')